In [ ]:
!wget --quiet -O UTKFace.tar.gz \
    https://huggingface.co/datasets/py97/UTKFace-Cropped/resolve/main/UTKFace.tar.gz

!mkdir -p UTKFace
!tar --strip-components=1 -xzf UTKFace.tar.gz -C UTKFace


In [2]:
import os
import requests
import tarfile
from tqdm import tqdm

# Create directory if it doesn't exist
os.makedirs("UTKFace", exist_ok=True)

# Download the dataset
url = "https://huggingface.co/datasets/py97/UTKFace-Cropped/resolve/main/UTKFace.tar.gz"
response = requests.get(url, stream=True)
total_size = int(response.headers.get('content-length', 0))

# Download with progress bar
with open("UTKFace.tar.gz", "wb") as f:
    with tqdm(total=total_size, unit='B', unit_scale=True, desc="Downloading UTKFace dataset") as pbar:
        for data in response.iter_content(chunk_size=4096):
            f.write(data)
            pbar.update(len(data))

# Extract the dataset
print("Extracting dataset...")
with tarfile.open("UTKFace.tar.gz", "r:gz") as tar:
    tar.extractall(path="UTKFace")

# Clean up the tar file
os.remove("UTKFace.tar.gz")
print("Download and extraction complete!")


C:\Users\lenovo\AppData\Local\Temp\ipykernel_14056\651285805.py:24: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path="UTKFace")


Extracting dataset...
Download and extraction complete!


In [11]:
import os
import random
from glob import glob
from typing import List, Optional

import numpy as np
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models
from torchvision.transforms import (
    Compose, ToTensor, Resize, Normalize,
    RandomHorizontalFlip, RandomRotation
)
from sklearn.metrics import accuracy_score, mean_absolute_error


In [ ]:
# ===============================
# 1. CONFIGURATION & HYPERPARAMS
# ===============================

DATA_DIR        = "UTKFace/"
IMAGE_SIZE      = 224
BATCH_SIZE      = 256
NUM_EPOCHS      = 40
LR              = 1e-4
AGE_LOSS_WEIGHT = 0.01
MAX_SAMPLES     = 20000  # limit total images
VAL_SPLIT       = 0.15
TEST_SPLIT      = 0.15
SEED            = 42

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# reproducibility
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

print(f"Using device: {DEVICE}")


In [13]:
# ===========================
# 2. UTKFace Utilities
# ===========================

def list_utkface_paths(
    image_dir: str,
    max_samples: Optional[int] = None,
    seed: Optional[int] = None
) -> List[str]:
    """
    Gather all valid UTKFace image paths, shuffle once, and optionally limit.
    """
    all_paths = glob(os.path.join(image_dir, "*.jpg"))
    valid_paths = []
    for p in all_paths:
        fn = os.path.basename(p)
        parts = fn.split("_")
        if len(parts) >= 4:
            try:
                age = int(parts[0])
                gender = int(parts[1])
                if 0 <= age <= 116 and gender in (0, 1):
                    valid_paths.append(p)
            except ValueError:
                continue

    if seed is not None:
        random.Random(seed).shuffle(valid_paths)
    else:
        random.shuffle(valid_paths)

    if max_samples is not None and len(valid_paths) > max_samples:
        valid_paths = valid_paths[:max_samples]

    print(f"Found {len(valid_paths)} valid UTKFace images")
    return valid_paths


class UTKFaceDataset(Dataset):
    """
    UTKFace Dataset: returns a dict with:
      - 'images': Tensor[C,H,W]
      - 'age':   float32
      - 'gender': int64
    """
    def __init__(self, paths: List[str], transform: Optional[Compose] = None):
        self.paths = paths
        self.transform = transform

        # Always apply resize → tensor → normalize
        self.normalize = Compose([
            Resize((IMAGE_SIZE, IMAGE_SIZE)),
            ToTensor(),
            Normalize(mean=[0.485, 0.456, 0.406],
                      std=[0.229, 0.224, 0.225]),
        ])

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]
        img = Image.open(path).convert("RGB")

        fn = os.path.basename(path)
        age = float(fn.split("_")[0])
        gender = int(fn.split("_")[1])

        if self.transform:
            img = self.transform(img)
        img = self.normalize(img)

        return {
            "images": img,
            "age": torch.tensor(age, dtype=torch.float32),
            "gender": torch.tensor(gender, dtype=torch.long)
        }


In [14]:
# ==============================
# 3. Multi-Task ResNet Model
# ==============================

class MultiTaskResNet(nn.Module):
    def __init__(self):
        super().__init__()
        # Load pretrained ResNet-18 and strip off its final layer
        backbone = models.resnet18(pretrained=True)
        self.feature_extractor = nn.Sequential(*list(backbone.children())[:-1])
        hidden = backbone.fc.in_features  # should be 512

        # Gender head: 2-way classification
        self.gender_head = nn.Sequential(
            nn.Linear(hidden, hidden // 4),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden // 4, 2)
        )

        # Age head: single-output regression
        self.age_head = nn.Sequential(
            nn.Linear(hidden, hidden // 4),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden // 4, 1)
        )

        self.ce_loss = nn.CrossEntropyLoss()
        self.mse_loss = nn.MSELoss()

    def forward(self, x, gender_labels=None, age_labels=None):
        # x: [B, 3, H, W]
        feats = self.feature_extractor(x)        # [B, 512, 1, 1]
        feats = feats.view(feats.size(0), -1)    # [B, 512]

        gender_logits = self.gender_head(feats)  # [B, 2]
        age_pred = self.age_head(feats).squeeze(-1)  # [B]

        losses = {}
        if gender_labels is not None and age_labels is not None:
            g_loss = self.ce_loss(gender_logits, gender_labels)
            a_loss = self.mse_loss(age_pred, age_labels)
            losses["gender_loss"] = g_loss
            losses["age_loss"]   = a_loss
            losses["loss"]       = g_loss + AGE_LOSS_WEIGHT * a_loss

        return gender_logits, age_pred, losses


In [15]:
# ======================
# 4. Training / Eval
# ======================

def train_one_epoch(model, loader, optimizer):
    model.train()
    running_loss = 0.0
    for batch in loader:
        imgs   = batch["images"].to(DEVICE)
        genders= batch["gender"].to(DEVICE)
        ages   = batch["age"].to(DEVICE)

        optimizer.zero_grad()
        _, _, losses = model(imgs, genders, ages)
        loss = losses["loss"]
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    return running_loss / len(loader)


def evaluate(model, loader):
    model.eval()
    all_g_true, all_g_pred = [], []
    all_a_true, all_a_pred = [], []

    with torch.no_grad():
        for batch in loader:
            imgs   = batch["images"].to(DEVICE)
            genders= batch["gender"].to(DEVICE)
            ages   = batch["age"].to(DEVICE)

            g_logits, a_pred, _ = model(imgs)
            g_preds = g_logits.argmax(dim=1)

            all_g_true.extend(genders.cpu().tolist())
            all_g_pred.extend(g_preds.cpu().tolist())
            all_a_true.extend(ages.cpu().tolist())
            all_a_pred.extend(a_pred.cpu().tolist())

    gender_acc = accuracy_score(all_g_true, all_g_pred)
    age_mae    = mean_absolute_error(all_a_true, all_a_pred)
    return gender_acc, age_mae


In [16]:
# ======================
# 5. Main
# ======================

def main():
    # 1. Gather & split paths
    paths = list_utkface_paths(DATA_DIR, max_samples=MAX_SAMPLES, seed=SEED)
    n_total = len(paths)
    n_test  = int(TEST_SPLIT  * n_total)
    n_val   = int(VAL_SPLIT   * n_total)
    n_train = n_total - n_val - n_test

    train_paths = paths[:n_train]
    val_paths   = paths[n_train:n_train+n_val]
    test_paths  = paths[n_train+n_val:]

    print(f"Split → train: {len(train_paths)}, val: {len(val_paths)}, test: {len(test_paths)}")

    # 2. Transforms & Datasets
    train_transform = Compose([
        RandomHorizontalFlip(0.5),
        RandomRotation(10),
    ])
    train_ds = UTKFaceDataset(train_paths, transform=train_transform)
    val_ds   = UTKFaceDataset(val_paths,   transform=None)
    test_ds  = UTKFaceDataset(test_paths,  transform=None)

    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  drop_last=True)
    val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False)
    test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False)

    # 3. Model, optimizer, scheduler
    model     = MultiTaskResNet().to(DEVICE)
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-2)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

    best_acc = 0.0
    for epoch in range(1, NUM_EPOCHS+1):
        train_loss = train_one_epoch(model, train_loader, optimizer)
        val_acc, val_mae = evaluate(model, val_loader)
        scheduler.step()

        print(f"Epoch {epoch:2d} | "
              f"Train Loss: {train_loss:.4f} | "
              f"Val Gender Acc: {val_acc*100:5.2f}% | "
              f"Val Age MAE: {val_mae:5.2f}")

        # checkpoint
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), "best_utkface_resnet.pt")
            print("  → New best model saved.")

    # 4. Final test
    model.load_state_dict(torch.load("best_utkface_resnet.pt", map_location=DEVICE))
    test_acc, test_mae = evaluate(model, test_loader)
    print(f"\n*** FINAL TEST → Gender Acc: {test_acc*100:.2f}% | Age MAE: {test_mae:.2f} yrs ***")


In [ ]:
main()